In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use("seaborn")
sns.set(style="white")

## Lectura y exploración

### La idea es generar un perfil de las peliculas y los usuarios.
### Empiezo por las peliculas y con su genero, para los generos 

In [2]:
df_movies = pd.read_csv("./data/movie_genres.csv")
df_genres = pd.read_csv("./data/movies_genres.csv")

FileNotFoundError: [Errno 2] File ./data/movies_genres.csv does not exist: './data/movies_genres.csv'

In [5]:
# Ahora tengo el userID
df_train.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [5]:
df_train.head()

,ID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,608,5.0,17,9,1997,23,40,20
1,2,141,4.0,17,9,1997,23,41,58
2,3,1073,4.0,17,9,1997,23,41,6
3,4,356,4.0,17,9,1997,23,43,44
4,5,1198,5.0,17,9,1997,23,44,35


## Filtrado y generacion de datos

In [69]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
full_x = df_train.drop(['rating', 'ID'], axis=1)
full_y = df_train[['rating']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)

In [70]:
print("train: ",len(X_train),len(y_train))
print("test:  ",len(X_test),len(y_test))

train:  539062 539062
test:   231027 231027


In [71]:
# Creo regresion lineal
model = LinearRegression()

# Entreno el modelo usando el conjunto de TRAIN
model.fit(X_train, y_train)

# Hago predicciones usando el TEST 
prediction = model.predict(X_test)

In [72]:
#ECM
print('Mean squared error: %.2f' % mean_squared_error(y_test, prediction))

Mean squared error: 1.01


## Modelo completo

In [73]:
df_x = df_test.drop(['rating', 'ID'], axis=1)
#df_y = df_test[['rating']]

In [74]:
# Creo regresion lineal
modelFull = LinearRegression()

# Entreno el modelo usando el conjunto de TRAIN
model.fit(X_train, y_train)

# Hago predicciones usando el TEST 
predFull = model.predict(df_x)

In [119]:
print(len(df_test.ID),len(predFull))

85509 85509


In [120]:
submit = pd.DataFrame({'ID':df_test.ID})
submit['rating'] = pd.DataFrame(np.around(predFull, 2))
submit.head()

,ID,rating
0,9,3.48
1,18,3.45
2,34,3.45
3,39,3.45
4,50,3.48


In [121]:
len(submit)

85509

In [117]:
submit.to_csv('./data/out/ignacio_submit1.csv', index=False)

## Ejemplo  Whoosh
### https://whoosh.readthedocs.io/en/latest/intro.html

In [32]:
import os
os.mkdir('index_dir')

In [33]:
import whoosh as wh
from whoosh import fields
from whoosh import index
from whoosh import query

import pandas as pd

In [34]:
schema = wh.fields.Schema(
    id=wh.fields.ID(stored=True),
    genres=wh.fields.KEYWORD(stored=True, lowercase=True, commas=True)
)

In [35]:
df_movies_genres = pd.read_csv('./data/movie_genres.csv')

In [38]:
ix = wh.index.create_in("index_dir", schema)
writer = ix.writer()

for mid in df_movies_genres.movieID.unique():    
    writer.add_document(id=str(mid), genres=u",".join(df_movies_genres.loc[df_movies_genres["movieID"]==mid, "genre"].tolist()))

writer.commit()

In [39]:
searcher = ix.searcher()

query = wh.query.Term("genres", "action") & wh.query.Term("genres", "drama")
results  = searcher.search(query, limit=None)
for r in results:
    print(r)

'}>
<Hit {'genres': 'Action,Drama,Thriller,Western', 'id': '3681'}>
<Hit {'genres': 'Action,Crime,Drama,Thriller', 'id': '3682'}>
<Hit {'genres': 'Action,Comedy,Crime,Drama', 'id': '3716'}>
<Hit {'genres': 'Action,Crime,Drama,Thriller', 'id': '3729'}>
<Hit {'genres': 'Action,Drama,War', 'id': '3753'}>
<Hit {'genres': 'Action,Crime,Drama,Thriller', 'id': '3761'}>
<Hit {'genres': 'Action,Adventure,Drama', 'id': '3805'}>
<Hit {'genres': 'Action,Adventure,Drama', 'id': '3907'}>
<Hit {'genres': 'Action,Drama,Thriller', 'id': '3946'}>
<Hit {'genres': 'Action,Crime,Drama,Thriller', 'id': '3947'}>
<Hit {'genres': 'Action,Adventure,Drama,Fantasy', 'id': '3996'}>
<Hit {'genres': 'Action,Drama,War,Western', 'id': '4042'}>
<Hit {'genres': 'Action,Crime,Drama', 'id': '4064'}>
<Hit {'genres': 'Action,Crime,Drama', 'id': '4065'}>
<Hit {'genres': 'Action,Comedy,Crime,Drama', 'id': '4085'}>
<Hit {'genres': 'Action,Adventure,Drama,Thriller', 'id': '4142'}>
<Hit {'genres': 'Action,Drama', 'id': '4163'}>


In [40]:
PERFIL = {
    "action": 0.1,
    "comedy": 0.9,
}

searcher = ix.searcher()
query = wh.query.Term("genres", "action") | wh.query.Term("genres", "comedy")
results  = searcher.search(query, limit=None, terms=True)
for r in results:
    score = 0
    for _, g in r.matched_terms():
        score += PERFIL[g.decode('utf-8')]
    print(r, score)

'genres': 'Comedy,Drama', 'id': '50583'}> 0.9
<Hit {'genres': 'Comedy,Drama,Romance', 'id': '50685'}> 0.9
<Hit {'genres': 'Comedy,Drama,Romance', 'id': '50792'}> 0.9
<Hit {'genres': 'Action,Crime,Drama,Thriller', 'id': '50794'}> 0.1
<Hit {'genres': 'Adventure,Comedy', 'id': '50798'}> 0.9
<Hit {'genres': 'Comedy,Drama,Romance', 'id': '50802'}> 0.9
<Hit {'genres': 'Comedy,Romance', 'id': '50806'}> 0.9
<Hit {'genres': 'Comedy,Drama', 'id': '50842'}> 0.9
<Hit {'genres': 'Animation,Children,Comedy,Fantasy', 'id': '50872'}> 0.9
<Hit {'genres': 'Comedy', 'id': '50970'}> 0.9
<Hit {'genres': 'Action,Adventure,Drama,War', 'id': '51007'}> 0.1
<Hit {'genres': 'Action,Fantasy,Thriller', 'id': '51077'}> 0.1
<Hit {'genres': 'Comedy,Romance', 'id': '51082'}> 0.9
<Hit {'genres': 'Comedy,Romance', 'id': '51084'}> 0.9
<Hit {'genres': 'Comedy', 'id': '51088'}> 0.9
<Hit {'genres': 'Comedy,Drama,Romance', 'id': '51094'}> 0.9
<Hit {'genres': 'Action,Adventure', 'id': '51277'}> 0.1
<Hit {'genres': 'Action,Sci